In [1]:
#Importing required headers.
import pandas as pd
import numpy as np
import requests, base64, json, logging

In [2]:
#Reading dataset.
dataset = pd.read_csv('../MSD10k.csv')

#Dropping irrelevant attributes.
dataset = dataset.drop(['artist_7digitalid', 'artist_mbid', 'artist_mbtags',\
                        'artist_mbtags_count', 'artist_playmeid', 'audio_md5',\
                        'danceability', 'energy', 'release_7digitalid', 'similar_artists',\
                        'song_id', 'track_7digitalid'], axis=1)
dataset = dataset.drop(['Unnamed: 0', 'transfer_note', 'analysis_sample_rate'], axis=1)
dataset = dataset.drop(0)

#Cleaning attributes.
#Replace null with numpy's nan.
dataset['artist_latitude'] = dataset['artist_latitude'].replace(b'', np.nan)

In [3]:
#Getting feel of dataset.
dataset.head()

,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,artist_terms,artist_terms_freq,artist_terms_weight,...,song_hotttnesss,start_of_fade_out,tatums_confidence,tatums_start,tempo,time_signature,time_signature_confidence,title,track_id,year
1,0.630630,0.417500,b'ARMJAGH1187FB546F3',35.14968,"b'Memphis, TN'",-90.04892,b'The Box Tops',[b'blue-eyed soul' b'pop rock' b'blues-rock' b...,[1. 0.89319999 0.78606029 0.74638538 0...,[1. 0.8459884 0.83068957 0.79929112 0...,...,NaN,137.915,[0.969 0.929 0.897 0.871 0.856 0.845 0.839 0.8...,[ 0.20627 0.45904 0.71054 0.96834 1.2...,121.274,4.0,0.384,b'Soul Deep',b'TRAAABD128F429CF47',1969.0
2,0.487357,0.343428,b'ARKRRTF1187B9984DA',NaN,b'',NaN,b'Sonora Santanera',[b'salsa' b'cumbia' b'tejano' b'ranchera' b'la...,[1. 0.94223907 0.94223907 0.94223907 0...,[1. 0.95825785 0.95825785 0.95825785 0...,...,NaN,172.304,[0.482 0.676 0.627 0.549 0.279 0.264 0.272 0.2...,[ 0.42132 0.73152 1.06609 1.39732 1.7...,100.070,1.0,0.000,b'Amor De Cabaret',b'TRAAADZ128F9348C2E',0.0
3,0.630382,0.454231,b'AR7G5I41187FB4CE6C',NaN,"b'London, England'",NaN,b'Adam Ant',[b'pop rock' b'new wave' b'dance rock' b'rock'...,[0.98858386 0.96725046 0.82060405 1. 0...,[1. 0.96369721 0.926773 0.89120673 0...,...,NaN,217.124,[0.601 0.556 0.523 0.49 0.466 0.44 0.428 0.4...,[ 0.56254 0.81002 1.05749 1.30621 1.5...,119.293,4.0,0.000,b'Something Girls',b'TRAAAEF128F4273421',1982.0
4,0.651046,0.401724,b'ARXR32B1187FB57099',NaN,b'',NaN,b'Gob',[b'pop punk' b'ska punk' b'breakcore' b'altern...,[0.8872883 0.79020373 0.79020373 0.79020373 0...,[1. 0.96090574 0.95923662 0.94674975 0...,...,0.604501,198.699,[1. 0.98 0.932 0.87 0.82 0.793 0.768 0.7...,[1.3576000e-01 3.6918000e-01 5.9914000e-01 8.3...,129.738,4.0,0.562,b'Face the Ashes',b'TRAAAFD128F92F423A',2007.0
5,0.535293,0.385471,b'ARKFYS91187B98E58F',NaN,b'',NaN,b'Jeff And Sheri Easter',[b'southern gospel' b'bluegrass' b'gospel' b'b...,[1. 0.87767844 0.5756383 0.47221964 0...,[1. 0.88934906 0.66082789 0.58984477 0...,...,NaN,254.270,[0.136 0.127 0.113 ... 0. 0. 0. ],[ 0.53929 0.74856 0.95987 ... 258.8164 2...,147.782,3.0,0.454,b'The Moon And I (Ordinary Day Album Version)',b'TRAAAMO128F1481E7F',0.0


In [4]:
#Getting feel of dataset.
#Please read carefully.
dataset.describe()

,artist_familiarity,artist_hotttnesss,artist_latitude,artist_longitude,duration,end_of_fade_in,key,key_confidence,loudness,mode,mode_confidence,song_hotttnesss,start_of_fade_out,tempo,time_signature,time_signature_confidence,year
count,9995.000000,9999.000000,3742.000000,3742.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,5647.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000
mean,0.565454,0.385551,37.157357,-63.933358,238.509476,0.758667,5.276528,0.449545,-10.485597,0.691169,0.477768,0.342776,229.976569,122.918521,3.564756,0.509910,934.798080
std,0.160169,0.143654,15.598494,50.508223,114.143054,1.868038,3.554007,0.274973,5.400054,0.462034,0.191257,0.247218,112.201291,35.184830,1.266295,0.373418,996.656656
min,0.000000,0.000000,-41.280540,-162.436500,1.044440,0.000000,0.000000,0.000000,-51.643000,0.000000,0.000000,0.000000,1.044000,0.000000,0.000000,0.000000,0.000000
25%,0.467602,0.325227,33.748310,-93.291560,176.025670,0.000000,2.000000,0.225000,-13.163500,0.000000,0.360000,0.000000,168.856000,96.969500,3.000000,0.097500,0.000000
50%,0.563584,0.380728,38.997920,-80.237420,223.059140,0.199000,5.000000,0.469000,-9.380000,1.000000,0.487000,0.360371,213.879000,120.162000,4.000000,0.551000,0.000000
75%,0.668020,0.453866,43.648560,-9.401020,276.375060,0.421000,8.000000,0.659000,-6.532000,1.000000,0.606000,0.537504,266.292000,144.014500,4.000000,0.864000,2000.000000
max,1.000000,1.082503,69.651020,174.767140,1819.767710,43.119000,11.000000,1.000000,0.566000,1.000000,1.000000,1.000000,1813.426000,262.828000,7.000000,1.000000,2010.000000


In [5]:
#Getting feel of dataset.
dataset.columns.values

array(['artist_familiarity', 'artist_hotttnesss', 'artist_id',
       'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'artist_terms', 'artist_terms_freq',
       'artist_terms_weight', 'bars_confidence', 'bars_start',
       'beats_confidence', 'beats_start', 'duration', 'end_of_fade_in',
       'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence',
       'release', 'sections_confidence', 'sections_start',
       'segments_confidence', 'segments_loudness_max',
       'segments_loudness_max_time', 'segments_loudness_start',
       'segments_pitches', 'segments_start', 'segments_timbre',
       'song_hotttnesss', 'start_of_fade_out', 'tatums_confidence',
       'tatums_start', 'tempo', 'time_signature',
       'time_signature_confidence', 'title', 'track_id', 'year'],
      dtype=object)

In [6]:
#Storing results.
dataset.to_csv("../Cleaned_MSD10k.csv")